<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.955 · Models avançats de mineria de dades · PAC1</p>
<p style="margin: 0; text-align:right;">2021-1 · Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PAC 1: Preparació de dades

En aquesta activitat pràctica veurem com aplicar diferents tècniques per a la càrrega i preparació de dades:

 <ol start="1">
  <li>Càrrega del conjunt de dades</li>
  <li>Anàlisi de les dades
  <br>2.1 Anàlisi estadístic bàsic
  <br>2.2 Anàlisi exploratori de les dades
  <li>Reducció de la dimensionalitat</li>
  <li>Entrenament i test</li>
</ol>

**Important:** Cada un dels exercicis pot suposar diversos minuts d'execució, de manera que el lliurament s'ha de fer en format notebook i en format html, on es vegi el codi, els resultats i comentaris de cada exercici. Es pot exportar el notebook a html des del menú File $\to$ Download as $\to$ HTML.

**Important:** Hi ha un tipus de cel·la especial per albergar text. Aquest tipus de cel·la us serà molt útil per respondre a les diferents preguntes teòriques plantejades al llarg de cada PAC. Per canviar el tipus de cel·la a aquest tipus, al menú: Cell $\to$ Cell Type $\to$ Markdown.
   
Per a això, necessitarem les següents llibreries:

In [1]:
from six import StringIO

from IPython.display import Image 
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import pydotplus
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('display.max_columns', None)

# 1. Càrrega del conjunt de dades (1 punt)

En primer lloc, haureu de carregar el conjunt de dades Breast Cancer Wisconsin (més informació a https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). Aquest es pot descarregar d'Internet o es pot carregar directament usant una llibreria de ``scikit-learn``, que incorpora un conjunt de datasets molt coneguts i emprats per mineria de dades i machine learning (https://scikit-learn.org/stable /datasets.html).

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> carregueu el conjunt de dades "Breast Cancer Wisconsin" i mostreu:
  <br> - El nombre i nom dels atributs (variables que podrien ser usades per a predir la resposta "Diagnosi")
  <br> - El nombre de files obtingudes
  <br> - Verifiqueu si hi ha o no dades que falten ( "missing values") i en quines columnes
</div>

**Suggeriment**: si feu servir ``sklearn`` (sklearn.datasets.load_breast_cancer), exploreu les diferents keys de l'objecte obtingut.
    
**Suggeriment**: potser us resulta útil passar les dades (atributs + target) a un dataframe de ``pandas``.

In [3]:
brcancer = datasets.load_breast_cancer()

print("elements of breastcancer: ", brcancer.keys())

elements of breastcancer:  dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [4]:
print(brcancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [5]:
X = brcancer.data
y = brcancer.target

print("Number of samples: ", X.shape[0])
print("Number of features: ", X.shape[1])

print("number of samples of target: ", y.shape[0])

feat_names = brcancer.feature_names

print("NUmber of features with names: ", len(feat_names))
print("Features names: ", feat_names)

Number of samples:  569
Number of features:  30
number of samples of target:  569
NUmber of features with names:  30
Features names:  ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [7]:
df_brcancer = pd.concat([pd.DataFrame(X, columns=feat_names), pd.DataFrame(y, columns=['Diagnosis'])], axis=1)
print("Number of rows: ", df_brcancer.shape[0], "Number of columns: ", df_brcancer.shape[1])

df_brcancer.head(5)

Number of rows:  569 Number of columns:  31


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


# 2. Anàlisi de les dades (3 punts)

## 2.1 Anàlisi estadístic bàsic

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> feu un anàlisi estadístic bàsic:
    <br> - Il·lustreu gràficament la freqüència d'aparició de les variables categòriques, indicant el nombre.
    <br> - Calculeu per a cada atribut numèric del conjunt de dades, el seu valor mitjà i desviació estàndard. 
</div>

**Consell**: podeu fer servir la llibreria ``pandas`` i les seves funcions ``describe`` i ``value_counts``.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Anàlisi:</strong> comenteu els resultats.
</div>

## 2.2 Anàlisi exploratori de les dades

En aquest exercici anem a explorar la relació d'alguns dels atributs numèrics amb la variable resposta ("Diagnosi"), tant gràficament com quantitativament i analitzarem les diferents correlacions. Per començar, seleccioneu els 4 atributs que voleu explorar. Si heu fet servir els noms de columnes proporcionats per ``sklearn``, una llista de possibles noms d'atributs serien:

In [ ]:
feats_to_explore = ['mean radius', 'mean texture', 'mean smoothness', 'mean symmetry']

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> utilitzant una llibreria gràfica, com ara "matplotlib", feu un gràfic del histograma de valors per a cada un dels atributs seleccionats, separats pels valors de classe de la resposta ("Diagnosi").

La finalitat és observar com es distribueix cada un dels atributs en funció de la valor de classe que prenen, per poder identificar de forma visual i ràpida si alguns atributs permeten diferenciar de forma clara els diferents diagnòstics dels tumors.
</div>

**Consell**: podeu fer servir el paràmetre ``alpha`` en els gràfics perquè s'aprecien els tres histogrames.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Anàlisi:</strong> mirant els gràfics, quins atributs creus que tenen valors clarament diferents per a diagnòstics benignes i malignes? Quins atributs no?</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> per a cada un dels atributs representats en l'exercici anterior, mostreu la mitjana i la desviació estàndard per a cada valor de classe de la resposta.
</div>
    
La finalitat és verificar numèricament les diferències identificades anteriorment de forma visual.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Anàlisi:</strong> mirant als valors mitjans i desviacions estàndard calculades, creus que són significatives les diferències per diagnòstics benignes i malignes?</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> calculeu i mostreu la correlació entre totes les variables del conjunt de dades.
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> representeu gràficament les relacions entre totes les parelles de les variables "mean" del conjunt de dades.
    
La finalitat d'aquest exercici és poder observar i analitzar les correlacions de manera gràfica entre algunes de les variables.
</div>

**Consell**: podeu utilitzar la funció ``pairplot`` de la llibreria ``Seaborn``.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> determineu quines són les variables amb una correlació superior al 90% i creeu un nou conjunt de dades (dataframe) eliminant aquestes variables.</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Anàlisi:</strong> mirant les correlacions, quines variables són les que tenen una correlació més forta i han estat eliminades? Quin efecte pot tenir en el resultat final eliminar aquestes variables?
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Bonus:</strong> Quin test estadístic podries fer servir per comparar les distribucions dels valors dels atributs quan els diagnòstics són benignes o malignes? Fes-lo servir per avaluar quins atributs (d'entre tots els del conjunt de dades) tenen valors més semblants a tumors benignes i malignes. Interpreta el resultat.
</div>

# 3. Reducció de dimensionalitat (3 punts)

En aquest exercici haureu aplicar mètodes de reducció de dimensionalitat al conjunt original de dades. L'objectiu és reduir el conjunt d'atributs a un nou conjunt amb menys dimensions.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> apliqueu el mètode de reducció de dimensionalitat Principal Component Analysis (PCA) per reduir a 2 dimensions:
<br>- El conjunt de dades original
<br>- El conjunt de dades a el qual se li han extret les variables fortament correlacionades
</div>

**Nota**: Tingueu cura i no inclogueu la variable objectiu "Diagnosi" a la reducció de dimensionalitat. Volem poder explicar la variable objectiu en funció de la resta de variables reduïdes a dues dimensions.

**Consell**: no cal que programeu els algoritmes, podeu fer ús de les implementacions disponibles a la llibreria ``scikit-learn``.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> genereu un gràfic (en 2D) amb el resultat del PCA usant colors diferents per a cada classe de la resposta, amb l'objectiu de visualitzar si és possible separar les classes emprant aquest mètode.</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Anàlisi:</strong> comenteu el resultat.</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Pregunta:</strong> Quin altre mètode o mètodes de reducció de dimensionalitat es podrien aplicar?</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> escull un mètode de reducció i genera el gràfic (2D) amb el resultat.</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Anàlisi:</strong> en comparació amb el PCA, quin mètode ha resultat més efectiu per a visualitzar les dues classes de diagnòstic de forma separada? Quan és més adequat fer servir PCA o l'altre mètode escollit?</div>

# 4. Entrenament i test (3 punts)

En aquest últim exercici, aplicarem un mètode d'aprenentatge supervisat, concretament el classificador Decision Tree (un arbre de decisió), per predir el diagnòstic de càncer de mama (tumor benigne o maligne) i avaluar la precisió obtinguda amb el model. 

Per a això farem servir:
- El conjunt de dades original amb tots els atributs
- El conjunt de dades reduït a només 2 atributs amb PCA
- El conjunt de dades sense les variables fortament correlacionada i reduït a només 2 atributs amb el mètode PCA

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> usant el conjunt de dades original:
  - Dividiu el dataset en train i test.
  - Definiu un model Decision Tree (fixant max_depth=5 per mantenir el model simple).
  - Apliqueu validació creuada amb el model definit i el dataset de train (amb cv=5 ja és suficient).
  - Calculeu la mitjana i la desviació estàndard de la validació creuada.
</div>

**Notes**: 
- Per separar entre train i test podeu fer servir ``train_test_split`` de ``sklearn``.
- Per entrenar un model _decision tree_ podeu fer servir ``DecisionTreeClassifier`` de ``sklearn``.
- Per aplicar validació creuada podeu fer servir ``cross_val_score`` de ``sklearn``.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
    <strong>Exercici:</strong> feu el <i>fit</i> amb tot el conjunt de train, i genereu l'arbre de decisió corresponent.</div>

**Nota**: per dibuixar arbres de decisió, podeu consultar:
- https://towardsdatascience.com/visualizing-decision-trees-with-python-scikit-learn-graphviz-matplotlib-1c50b4aa68dc

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> repetiu el mateix procediment que en l'exercici anterior amb el dataset reduït a 2 dimensions amb PCA.</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> repetiu el mateix procediment que en l'exercici anterior amb el conjunt de dades sense les variables fortament correlacionades i reduït a només 2 atributs amb el mètode PCA.
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Anàlisi:</strong> Amb quines dades ha funcionat millor? Té sentit? És coherent amb els resultats que hem vist en l'exercici 3?
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> amb el millor model que hagueu obtingut:
  - Genereu prediccions sobre el dataset de test.
  - Calculeu la precisió de les prediccions obtingudes i la matriu de confusió associada.
</div>

**Nota**: per calcular la precisió i la matriu de confusió podeu fer servir les funcions dins del mòdul ``metrics`` de ``sklearn``.